Libraries

In [26]:
import os
from mistralai import Mistral
from dotenv import load_dotenv
import pandas as pd
from pymongo import MongoClient
load_dotenv()
import time

Env and client setup

In [28]:
api_key = os.environ["mistral_key"]
agent_id = os.environ["agent_id"]
db_url = os.environ["DB_URL"]
model = "open-mistral-nemo"
client = Mistral(api_key=api_key)
mongo_client = MongoClient(db_url)
db  = mongo_client.reddit
post_db = db['post_new']


ConfigurationError: The resolution lifetime expired after 21.608 seconds: Server Do53:83.169.184.33@53 answered The DNS operation timed out.; Server Do53:83.169.184.97@53 answered The DNS operation timed out.; Server Do53:83.169.184.33@53 answered The DNS operation timed out.; Server Do53:83.169.184.97@53 answered The DNS operation timed out.; Server Do53:83.169.184.33@53 answered The DNS operation timed out.; Server Do53:83.169.184.97@53 answered The DNS operation timed out.; Server Do53:83.169.184.33@53 answered The DNS operation timed out.; Server Do53:83.169.184.97@53 answered The DNS operation timed out.; Server Do53:83.169.184.33@53 answered The DNS operation timed out.; Server Do53:83.169.184.97@53 answered The DNS operation timed out.

Local Data import

In [9]:
rec = post_db.find()
rec = list(rec)
print(rec[0])

{'_id': ObjectId('6759c56c3031622e94ed9baf'), 'id': '1g1em48', 'title': 'Share your startup - quarterly post', 'selftext': "Share Your Startup - Q4 2023\n\n[**r/startups**](https://www.reddit.com/r/startups/) **wants to hear what you're working on!**\n\nTell us about your startup in a comment within this submission. Follow this template:\n=====================================================================================\n\n\u200b\n\n*   **Startup Name / URL**\n*   **Location of Your Headquarters**\n    *   Let people know where you are based for possible local networking with you and to share local resources with you\n*   **Elevator Pitch/Explainer Video**\n*   **More details:**\n    *   What life cycle stage is your startup at? (reference the stages below)\n    *   Your role?\n*   **What goals are you trying to reach this month?**\n    *   How could [r/startups](https://www.reddit.com/r/startups/) help?\n    *   Do **NOT** solicit funds publicly--this may be illegal for you to do s

In [10]:
rec[0]

{'_id': ObjectId('6759c56c3031622e94ed9baf'),
 'id': '1g1em48',
 'title': 'Share your startup - quarterly post',
 'selftext': "Share Your Startup - Q4 2023\n\n[**r/startups**](https://www.reddit.com/r/startups/) **wants to hear what you're working on!**\n\nTell us about your startup in a comment within this submission. Follow this template:\n=====================================================================================\n\n\u200b\n\n*   **Startup Name / URL**\n*   **Location of Your Headquarters**\n    *   Let people know where you are based for possible local networking with you and to share local resources with you\n*   **Elevator Pitch/Explainer Video**\n*   **More details:**\n    *   What life cycle stage is your startup at? (reference the stages below)\n    *   Your role?\n*   **What goals are you trying to reach this month?**\n    *   How could [r/startups](https://www.reddit.com/r/startups/) help?\n    *   Do **NOT** solicit funds publicly--this may be illegal for you to d

Basic Classification with Mistral

In [14]:
df=pd.DataFrame(rec)

In [17]:
df.shape

(7825, 9)

In [19]:

sample=post_db.find({'response':{'$exists':True}})

In [ ]:
print(list(sample[0]))

[]
